# SharePoint + Azure AI Search Integration Overview

This document describes the integration solutions for SharePoint with Azure AI Search Agentic Retrieval and **permission requirements**.

> 📝 **This document is an overview**. For specific code implementations, please refer to the three following Notebooks.

---

## ⚠️ Important: SharePoint Integration Requires Admin Authorization!

**SharePoint is Microsoft's most strictly permission-controlled service**. Integration solutions require admin authorization.

### 🔑 Permission Model Description

| Role | Can Do | Cannot Do |
|------|--------|-----------|
| **Developer** | Create App Registration, add API permissions, write code | Grant Admin Consent |
| **Global Admin** | Grant Admin Consent (click one consent button) | - |

**Simply put**: Developers prepare everything, then have the Global Admin **click a link to authorize**. After that, the code can run normally.

---

## 📁 Related Notebooks

| Notebook | Solution | Description |
|----------|----------|-------------|
| `03e_sharepoint_indexer.ipynb` | **Manual Indexer** | Manually create Data Source + Index + Indexer, full control |
| `03f_indexed_sharepoint_ks.ipynb` | **Indexed SP KS** | Auto-create entire pipeline with Embedding + image processing |
| `03h_remote_sharepoint_ks.ipynb` | **Remote SP KS** | Real-time query using user permissions |

---

## 🔄 Three Solutions Comparison

| Solution | Description | Required Authorization | Vector Search | User Permissions | Latency |
|----------|-------------|----------------------|---------------|------------------|---------|
| **Remote SP KS** | Real-time query using user permissions | M365 Copilot License | ❌ | ✅ | High |
| **Indexed SP KS** | Auto pipeline with Embedding | Admin Consent* | ✅ | ❌ | Low |
| **Manual Indexer** | Full control, manual management | Admin Consent* | ✅** | ❌ | Low |

> \* Admin Consent: After developer creates App Registration and adds `Sites.Read.All` permission, Global Admin clicks link to authorize  
> \*\* Manual Indexer requires additional Skillset configuration for vector search support

### How to Choose?

```
Need user-level permission control?
    ├─ Yes → Remote SharePoint KS (03h)
    └─ No → Need auto pipeline?
                ├─ Yes → Indexed SharePoint KS (03f)
                └─ No → Manual Indexer (03e)
```

---

## 📖 Detailed Description of Each Solution

### 1️⃣ Remote SharePoint Knowledge Source

**Notebook**: `03h_remote_sharepoint_ks.ipynb`

```
User Query → Pass User Token → SharePoint Search API → Real-time Results
```

| Feature | Description |
|---------|-------------|
| **How it works** | Query passes user's Entra ID Token, real-time query to SharePoint |
| **Data storage** | No storage, real-time query |
| **Permission model** | Respects user permissions, different users see different content |
| **Pros** | Real-time data, secure permissions, no pre-indexing required |
| **Cons** | Requires M365 Copilot License ($30/month), no vector search support, higher latency |

### 2️⃣ Indexed SharePoint Knowledge Source

**Notebook**: `03f_indexed_sharepoint_ks.ipynb`

```
SharePoint → Auto-create Data Source + Skillset + Index + Indexer → Pre-index → Query
```

| Feature | Description |
|---------|-------------|
| **How it works** | One-click auto-create complete pipeline with Embedding and image semanticization |
| **Data storage** | Pre-indexed to Azure AI Search |
| **Permission model** | All users with KB query permission see the same content |
| **Pros** | Simple configuration, vector search support, image processing support, low latency |
| **Cons** | Requires Admin Consent, deleting KS cascades delete all resources |

### 3️⃣ SharePoint Indexer (Manual)

**Notebook**: `03e_sharepoint_indexer.ipynb`

```
Manually create: Data Source → Index → Indexer → (Optional) Skillset
```

| Feature | Description |
|---------|-------------|
| **How it works** | Connect to SharePoint via App Registration (Client ID + Secret) |
| **Data storage** | Pre-indexed to custom Index |
| **Permission model** | All users with Index query permission see the same content |
| **Pros** | Full control of Index Schema, can share Index with other data sources, flexible configuration |
| **Cons** | Requires manual configuration of multiple components, requires Admin Consent |

---

## 🔐 Permission Details

### What Developers Need to Do (No Admin Permission Required)

1. **Create App Registration** (In Entra ID, regular users can do this)
2. **Add API Permission**: `Sites.Read.All` (Application type)
3. **Create Client Secret**
4. **Send Admin Consent link to Global Admin**

### What Global Admin Needs to Do

**Just click a link and consent to authorization**:

```
https://login.microsoftonline.com/{tenant-id}/adminconsent?client_id={app-id}
```

> 💡 This is a one-time operation. After authorization, the developer's code can run normally

### API Permission Description

| Permission | Type | Description | Who Authorizes |
|------------|------|-------------|----------------|
| `Sites.Read.All` | Application | Read all Site content | Global Admin clicks to authorize |
| `Sites.FullControl.All` | Application | Full control of all Sites | Global Admin clicks to authorize |
| `Sites.Selected` | Application | Access only specific Sites | SharePoint Admin + Graph API configuration |

> 💡 **Recommended**: Use `Sites.Read.All`, sufficient permissions and simple configuration

---

## 🔑 Knowledge Source Deletion Behavior

| Type | Auto-created Resources | Deletion Behavior |
|------|------------------------|-------------------|
| **Remote SP KS** | ❌ None | ✅ Only deletes KS itself |
| **Indexed SP KS** | ✅ Index + Indexer + Skillset + DataSource | ⚠️ **Cascade deletes all** |
| **Manual Indexer + Native KS** | ❌ None (manually created) | ✅ Only deletes KS itself |

> ⚠️ **Important**: If using Indexed SP KS, deleting Knowledge Source will delete the entire Index!
> If you need to keep the Index, use Manual Indexer + Native Index KS solution.

---

## 🚀 Alternative Solutions

If Admin Consent cannot be obtained, consider the following alternatives:

| Solution | Description | Permission Requirements |
|----------|-------------|------------------------|
| **Azure Blob Storage** | Sync documents to Blob, use Blob Knowledge Source | Storage permission (no Admin Consent needed) |
| **OneLake** | Sync SharePoint data via Fabric | Fabric permission |
| **OneDrive** | Personal OneDrive has lower permission requirements | User self-authorization |
| **Azure Files** | Sync documents to Azure Files | Storage permission (no Admin Consent needed) |

---

## 📚 Reference Documentation

- [SharePoint Indexer Official Documentation](https://learn.microsoft.com/en-us/azure/search/search-howto-index-sharepoint-online)
- [Remote SharePoint Knowledge Source](https://learn.microsoft.com/en-us/azure/search/agentic-knowledge-source-how-to-sharepoint-remote)
- [Indexed SharePoint Knowledge Source](https://learn.microsoft.com/en-us/azure/search/agentic-knowledge-source-how-to-sharepoint-indexed)